# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [1]:
# if version does not include DeepState, uncomment the following:
# !pip install git+https://github.com/awslabs/gluon-ts.git

In [2]:
import mxnet
mxnet.__version__

'1.4.1'

In [3]:
import gluonts
gluonts.__version__

'0.4.2'

In [4]:
import pprint
from functools import partial

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [5]:
def deepfactor(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    df = pd.DataFrame([output])

    return df

## m4_daily

In [24]:
results = pd.DataFrame()

for i in range(45, 47):
    print("Seed:", i)
    df = deepfactor(data="m4_daily", seed=i, epochs=500, batches=100)
    pprint(df)
    results = results.append(df)

results

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]INFO:root:Number of parameters in DeepFactorTrainingNetwork: 54286


Seed: 45


100%|██████████| 100/100 [00:02<00:00, 41.16it/s, avg_epoch_loss=5.54e+7]
INFO:root:Epoch[0] Elapsed time 2.431 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=55360024.091429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 48.46it/s, avg_epoch_loss=1.96e+7]
INFO:root:Epoch[1] Elapsed time 2.065 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=19579773.611429
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 42.49it/s, avg_epoch_loss=7.66e+6]
INFO:root:Epoch[2] Elapsed time 2.356 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=7657068.136429
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 43.90it/s, avg_epoch_loss=3.83e+6]
INFO:root:Epoch[3] Elapsed time 2.280 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3831596.449286
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 40.58it/s, avg_epoch_loss=1.92e+6]
INFO:root:Epoch[

100%|██████████| 100/100 [00:02<00:00, 42.34it/s, avg_epoch_loss=2.91e+4]
INFO:root:Epoch[36] Elapsed time 2.364 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=29089.654513
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 40.37it/s, avg_epoch_loss=1.59e+4]
INFO:root:Epoch[37] Elapsed time 2.481 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=15905.634254
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 48.34it/s, avg_epoch_loss=1.6e+4]
INFO:root:Epoch[38] Elapsed time 2.070 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=15950.431685
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 41.30it/s, avg_epoch_loss=3.68e+4]
INFO:root:Epoch[39] Elapsed time 2.425 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=36834.036211
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 44.36it/s, avg_epoch_loss=1.36e+4]
INFO:root:Epoch

100%|██████████| 100/100 [00:02<00:00, 47.20it/s, avg_epoch_loss=678]
INFO:root:Epoch[108] Elapsed time 2.123 seconds
INFO:root:Epoch[108] Evaluation metric 'epoch_loss'=677.636034
INFO:root:Epoch[109] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 43.30it/s, avg_epoch_loss=586]
INFO:root:Epoch[109] Elapsed time 2.311 seconds
INFO:root:Epoch[109] Evaluation metric 'epoch_loss'=586.159563
INFO:root:Epoch[110] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 42.64it/s, avg_epoch_loss=535]
INFO:root:Epoch[110] Elapsed time 2.347 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=534.661610
INFO:root:Epoch[111] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 39.60it/s, avg_epoch_loss=499]
INFO:root:Epoch[111] Elapsed time 2.527 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=498.786537
INFO:root:Epoch[112] Learning rate is 0.001
100%|██████████| 100/100 [00:02<00:00, 47.47it/s, avg_epoch_loss=314]
INFO:root:Epoch[112] Elapsed t

GluonTSDataError: Encountered invalid loss value! Try reducing the learning rate or try a different likelihood.